In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import datetime
import os.path
import sys
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
import pyfolio as pf
import datetime
import backtrader.feeds as btfeed
import pandas as pd
import io
# from strategies.crossover_9_21_with_swing import Crossover9_21WithSwing
from strategies.delta_spread import DeltaSpread
from option_data_feed import OptionDataFeed
import warnings; warnings.simplefilter('ignore')

all_option_data_october = pd.read_csv('./data/options_data/all_options_data.csv')
underyling_data = pd.read_csv('./data/SPY.csv', parse_dates=True, index_col=0)
underlying = bt.feeds.PandasData(dataname=underyling_data)

def create_data_feed(pd_data, series_name):
    s_buf = io.StringIO()
    pd_data.to_csv(s_buf)
    s_buf.seek(0)
    option_feed = OptionDataFeed(
        dataname=s_buf,
        datetime=8,
        high=None,
        low=None,
        open=12,
        close=12,
        volume=13,
        openinterest=14,
        dtformat=('%m/%d/%Y'),
        name=series_name)
    return option_feed

def add_options_data():
    option_types = ['call', 'put']
    option_expirations = all_option_data_october['expiration'].unique()
    option_strikes = all_option_data_october['strike'].unique()
    for type in option_types:
        is_type = all_option_data_october['type'] == 'call'
        filtered_by_type = all_option_data_october[is_type]
        for expiration in option_expirations:
            is_expiration = filtered_by_type['expiration'] == expiration
            filtered_by_type_expiration = filtered_by_type[is_expiration]
            for strike in option_strikes:
                is_strike = filtered_by_type_expiration['strike'] == strike
                filtered_by_type_expiration_strike = filtered_by_type_expiration[is_strike]
                if not filtered_by_type_expiration_strike.empty:
                    series_name = type + "_" + str(strike) + "_" + str(expiration)
                    single_option_data = create_data_feed(filtered_by_type_expiration_strike, series_name)
                    cerebro.adddata(single_option_data)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(DeltaSpread, printlog=True)

cerebro.adddata(underlying)
add_options_data()
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)
cerebro.broker.setcommission(commission=0.65, margin=33000, mult=100)

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
results = cerebro.run()
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
pf.create_full_tear_sheet(returns)